In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt 

In [ ]:
df1 = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')

In [ ]:
df1.head()

In [ ]:
df1.shape

In [ ]:
df1.info()

In [ ]:
df1.isnull().sum()

In [ ]:
df1.drop(columns=['keyword', 'location','id'], inplace = True)

In [ ]:
df1.head()

In [ ]:
import nltk

In [ ]:
nltk.download('punkt')

In [ ]:
df1['number_of_character'] = df1['text'].apply(lambda x : len(x))

In [ ]:
df1['number_of_words'] = df1['text'].apply(lambda x: len(nltk.word_tokenize(x)))

In [ ]:
df1['number_of_sentence'] = df1['text'].apply(lambda x: len(nltk.sent_tokenize(x)))

In [ ]:
df1.head()

In [ ]:
import seaborn as sns

In [ ]:
sns.heatmap(df1.corr(), annot = True)

In [ ]:
df1['target'].value_counts()

In [ ]:
sns.countplot(df1['target'], data = df1)

In [ ]:
from nltk.corpus import stopwords
len(stopwords.words('english'))

In [ ]:
import string

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def transform(text):
    text = text.lower()
    text = nltk.word_tokenize(text)
    y = []
    for i in text:
        if i.isalnum():
            y.append(i)
    text = y[:]#copying list
    y.clear()
    for i in text:
        if i not in stopwords.words('english') and i not in string.punctuation:
            y.append(i)
    text = y[:]
    y.clear()
    for i in text:
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
df1['transform_text'] = df1['text'].apply(transform)

In [ ]:
df1.head()

In [ ]:
from wordcloud import WordCloud

In [ ]:
wc = WordCloud(width = 500, height = 500, min_font_size = 10, background_color='white')

In [ ]:
spam_wordcloud = wc.generate(df1[df1['target'] == 1]['transform_text'].str.cat(sep = " "))

In [ ]:
plt.figure(figsize=(15,8))
plt.imshow(spam_wordcloud)

In [ ]:
spam_wordcloud = wc.generate(df1[df1['target'] == 0]['transform_text'].str.cat(sep = " "))

In [ ]:
plt.figure(figsize=(15,8))
plt.imshow(spam_wordcloud)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

In [ ]:
df2 = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

In [ ]:
df2.head()

In [ ]:
df2.drop(columns=['keyword', 'id', 'location'], inplace = True)

In [ ]:
df2.head()

In [ ]:
df2.isnull().sum()

In [ ]:
df2['transform_text'] = df2['text'].apply(transform)

In [ ]:
df2.head()

In [ ]:
X = cv.fit_transform(df1['transform_text']).toarray()

In [ ]:
y = df1['target'].values

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB

In [ ]:
gnb = GaussianNB()
mnb = MultinomialNB()
bnb = BernoulliNB()

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
gnb.fit(X_train, y_train)
pred1 = gnb.predict(X_test)

In [ ]:
print(classification_report(y_test, pred1))

In [ ]:
mnb.fit(X_train, y_train)
pred2 = mnb.predict(X_test)
print(classification_report(y_test, pred2))

In [ ]:
bnb.fit(X_train, y_train)
pred3 = bnb.predict(X_test)
print(classification_report(y_test, pred3))


In [ ]:
X_train = df1['transform_text']
y_train = df1['target']

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
from sklearn.svm import LinearSVC

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
text_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [ ]:
text_clf.fit(X_train, y_train)

In [ ]:
X_test = df2['transform_text']

In [ ]:
prdiction1 = text_clf.predict(X_test)

In [ ]:
submission = pd.DataFrame(prdiction1, columns = ['target'])

In [ ]:
submission.head(1)

In [ ]:
submission1 = pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')

In [ ]:
submission1.head()

In [ ]:
submission1.drop('target', axis = 1, inplace = True)

In [ ]:
submission1 = pd.concat([submission1, submission], axis = 1)

In [ ]:
submission1.head()

In [ ]:
submission1.to_csv('final_submit.csv', index = False)